In [ ]:
import torch
from energy_gap import energy_gap_3d
from adaptive.learner.learnerND import curvature_loss_function
from adaptive import notebook_extension, Runner, BlockingRunner, LearnerND
from tools.device_setter import multi_GPU_executor
notebook_extension()

In [ ]:
def main():
    try:
        torch.multiprocessing.set_start_method('spawn')
    except:
        pass

    print("Learner initialization")
    loss_function = curvature_loss_function()
    learner = LearnerND(energy_gap_3d, bounds = [(0, 1), (-1, 1), (0.2, 1.25)], loss_per_simplex = loss_function)
    #learner.load("/home/mamelz/DSKY/DSKY-Differentiable-spin-chain-on-skyrmion/data/sweep/saving.pickle")
    print("Runner initialization")

    tasks_per_gpu = 1
    gpus = range(1,16)
    executor = multi_GPU_executor(tasks_per_gpu = tasks_per_gpu, gpus = gpus)

    runner = Runner(learner, executor = executor, shutdown_executor = True, retries = 1000, goal = lambda l: l.loss() < 1e-12)
    runner.start_periodic_saving(save_kwargs=dict(fname = "data/sweep/sweep3d/alpha1_0.pickle"), interval = 900)
    if type(runner) != BlockingRunner:
        runner.live_info(update_interval = 1)

    return runner, learner

runner, learner = main()